In [8]:
# 导入
import torch
import sbibm
from sbi.inference import SNPE
import matplotlib.pyplot as plt

# 1. 任务与先验、模拟器
task = sbibm.get_task("two_moons")
prior = task.get_prior()
simulator = task.get_simulator()

# 2. 采样
num_simulations = 1000
theta = prior(num_simulations) 
x = simulator(theta)
print(f"Theta shape: {theta.shape}")
print(f"X shape: {x.shape}")


Theta shape: torch.Size([1000, 2])
X shape: torch.Size([1000, 2])


In [9]:
inference = SNPE(prior=prior)

# 训练 density estimator
density_estimator = inference.append_simulations(theta, x).train()

AssertionError: Prior must be a PyTorch Distribution. See FAQ 7 for more details or use
        `sbi.utils.user_input_checks.process_prior` for wrapping scipy and lists of
        independent priors.

# 先验猜测参数 sita

In [ ]:
posterior = inference.build_posterior(density_estimator)

# sbibm 自带观测obs（推荐，不要用 torch.zeros）
observation = task.get_observation(num_observation=1)
samples = posterior.sample((5000,), x=observation)
print("Posterior samples shape:", samples.shape)

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(samples[:,0], samples[:,1], alpha=0.25, s=4, label="Posterior Samples")
plt.xlabel(r"$\theta_1$")
plt.ylabel(r"$\theta_2$")
plt.title("Posterior Samples (Two Moons)")
plt.legend()
plt.grid(True)
plt.show()

# 

In [ ]:
from sbibm.visualisation import fig_posterior

fig = fig_posterior(task_name="two_moons", observation=1, samples=[samples])
fig.show()

# 执行推断

In [ ]:
reference = task.get_reference_posterior_samples(num_observation=1)
plt.figure(figsize=(6,6))
plt.scatter(reference[:,0], reference[:,1], color='g', s=4, alpha=0.3, label='Reference Posterior')
plt.scatter(samples[:,0], samples[:,1], color='r', s=3, alpha=0.3, label='SNPE Posterior')
plt.legend()
plt.title("Comparison: Reference Posterior vs SNPE Posterior")
plt.xlabel(r"$\theta_1$")
plt.ylabel(r"$\theta_2$")
plt.show()

# 可视化结果

In [ ]:
from sbibm.metrics import c2st
c2st_score = c2st(reference, samples)
print(f"C2ST score: {c2st_score:.3f}")

